Importing Libraries

In [5]:
%pip install --upgrade pip
%pip install requests pandas
%pip install lxml
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import table_utils
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
from datetime import datetime
import html

Defining HTML Content

In [ ]:
def fetch_bioc_xml(pmc_id: str, encoding: str = "ascii") -> str:
    """
    Uses API call to import PMC dataset paper in xml format and default ASCII encoding
    """
    url = f"https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/{pmc_id}/{encoding}"
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.text

def get_table_prompt(abstract, tables):
    """
    Using abstract and list of all tables, returns prompt to get whether the tables are related to toxicity study
    """
    table_prompt = f"<abstract>\n{abstract}\n</abstract>\n"
    for i in range(len(tables)):
        table_prompt += f"<table{i+1}>\n{tables[i]}\n</table{i+1}>\n"
    table_prompt += "<instructions>\nAbove is the abstract of a research paper and a series of tables that are in it. For each table, indicate whether it is a table indicating biomarker data from an animal toxicity experiment or not with a 0, representing no, or a 1, representing yes, in a list separated by commas within angle brackets like so: <0,0,1,0,1>. For every answer you give, double check which table it relates to and verify its source. Do not include any other text besides your answer and do not add any other text inside the list except for the requested information.\n</instructions>"
    return table_prompt

def get_formatting_prompt(table):
    """
    Returns prompt for formatting
    """
    formatting_prompt = f"<table>\n{table}\n</table>\n<instructions>\nAbove is a table representing animal toxicity data. Complete the following instructions in order and write the answer within angle brackets each on a new line. Do not include any other text besides your answer. Verify your information's source before adding it to your answer.\n1. Do the groups go vertically, where each treatment group occupies a separate column, or do they go horizontally, where each group occupies a row? Write <vertical> if vertical and <horizontal> if horizontal\n2. Are there any extra title columns on the left that should be combined? If so, write <n> where n is the number for how many columns that should be combined into a singular title column. n = 1 if there are no extra columns. \n3. In the numerical cells, are there any special characters that should be removed, such as *? If so, list any of them present in the brackets, separated by '|'.\n</instructions>\n<example>\n<horizontal>\n<1>\n<*,~>\n</example>"
    return formatting_prompt

def get_groups_prompt(paper, group_list):
    """
    Returns prompt for getting treatment group information
    Group Format: <group_name,animal_species,model_type,n,treatment_medication1,dose1,unit1,route1,freq1>
    """
    groups_prompt = f'<paper>\n{paper}\n</paper>\n<instructions>\nAbove is the methods section of a paper containing animal toxicity trials. For only the animal trials sections list the parameters of each of the groups (listed below), with their respective parameters in the format shown below separated by "|". For each piece of information, validate its source before adding it to your answer. If no treatments are performed, simply do not add any spaces for treatments. Do not add spaces between parameters and put each group on a new line. If you cannot find a certain piece of information, fill the blank with N/A. Do not include any text besides these groups and do not add any text inside the groups except for the requested information.\n<group_name|animal_species|model_type|xenografted_cells|sample_size|{{for each treatment i: treatment_medication_i|dose_i|unit_i|route_i|freq_i}}>\nGroup list: {group_list}\n</instructions>\n<example>\n<tumor control|mouse|CD-1|MDA-MB-231|5|DMSO|100|mul|iv|single dose>\n</example>'
    return groups_prompt

def get_biomarker_prompt(paper, table):
    """
    Returns prompt for getting toxicity biomarker information
    Biomarker Format: <biomarker_name,terminal_time,new_name,relevant_system,numerical_value,unit,summary_stat>
    """
    biomarkers_prompt = f"<paper>\n{paper}\n</paper>\n<instructions>\nAbove is the methods section of a paper and the following table is a processed version of the table from its animal toxicity study:\n<table>\n{table}\n</table>\nFor each of the columns, which represent toxicity biomarkers, list the details in this format:\n<column_name|terminal_time|biomarker_name|relevant_system|unit|summary_stat>\nThe column_name should be the original column name and the new_name should be just the biomarker, removing other information such as units, summary statistic type, and categories. In relevant_system, fill in one of skeletal, muscular, nervous, endocrine, cardiovascular, respiratory, digestive, reproductive, or N/A if the biomarker relates to none of these. For each piece of information, validate its source before adding it to your answer. Do not add spaces between parameters and put each group on a new line. If you cannot find a certain piece of information, fill the blank with N/A. ONLY include these treatment parameters and do not add any text inside them except for the requested information.\n</instructions>\n<example>\n<Organ Weights (g).Final Heart Weight.mean|2 weeks,Heart Weight|cardiovascular|g|mean></example>"
    return biomarkers_prompt


Enter PMC Here

In [8]:
PMC = "PMC10450773"

Downloading research paper

In [ ]:
xml = fetch_bioc_xml(PMC)
with open(f"papers/{PMC}.xml", "w", encoding="utf-8") as f:
    f.write(xml)


In [27]:
with open(f"papers/{PMC}.xml", "r", encoding="utf-8") as file:
    paper_xml = file.read()
soup = BeautifulSoup(paper_xml, "lxml-xml")
abstract_passages = [p for p in soup.find_all("passage") if p.find("infon", {"key": "section_type"}) and 
        p.find("infon", {"key": "section_type"}).text.strip().upper() == "METHODS"]
abstract_text_list = [p.find("text").get_text() for p in abstract_passages if p.find("text")]
abstract = "\n".join(abstract_text_list)
tables = [table_utils.single_html_table_to_markdown(html.unescape(p.get_text())) for p in soup.find_all("passage") if p.find("infon", {"key": "xml"})]
table_prompt = get_table_prompt(abstract, tables)
print(table_prompt)

<abstract>
Materials and methods
Plant materials
Fresh leaves of M. oleifera were collected locally from Institute of Bioscience (IBS), Universiti Putra Malaysia (UPM), between April to May 2017. The collected leaves were washed thoroughly to take away any dust and soil deposits. It was then air dried under shade at room temperature for 5 days to get rid of the moisture content. Thereafter, the dried leaves were ground to a fine powder (1.0 mm) using an electric Laboratory grinder. The dry matter content (DMC) and the percentage yield (biomass) of the plant's leaves were calculated as follows:
Botanical identification
The plant was botanically identified at the IBS, UPM, by Dr. Mohd Firdaus Ismail, where the kingdom, family and species of M. oleifera was confirmed. The voucher specimens were deposited in herbarium IBS, UPM (SK3168/17).
Extraction procedure
The extraction method described by was employed in this study with modifications. The dried powdered leaves were extracted using 70

In [28]:
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
version = os.getenv("OPENAI_API_VERSION")
model = os.getenv("OPENAI_MODEL")
api_key = os.getenv("OPENAI_API_KEY")

url = f"{endpoint}openai/deployments/{model}/chat/completions?api-version={version}"
print(url)

headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

table_payload = {
    "messages": [
        {"role": "user", "content": table_prompt}
    ],
    "temperature": 0,
    "max_tokens": 50,
}

#saving as {PMC}\n---\n{prompt}\n---\n{response}
table_response_json = requests.post(url, headers=headers, json=table_payload)
time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
with open(f"responses/{time}_table.txt", "w", encoding="utf-8") as file:
    file.write(f"{PMC}\n---\n{table_prompt}\n---\n{table_response_json.json()['choices'][0]['message']['content']}")
print(table_response_json.status_code)

table_bools = [table_response_json.json()['choices'][0]['message']['content'].strip()[1:-1].split(",")]

https://pharmacoinfo-openai-5.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-10-21
200


Fetches table manually extracted from research paper with path tables/{PMC}.txt

In [16]:
with open(f'tables/{PMC}.txt', 'r', encoding='utf-8') as f:
    html_content = f.read()

In [ ]:
xml_content = html.unescape(html_content)
markdown_table = table_utils.single_html_table_to_markdown(xml_content)

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
version = os.getenv("OPENAI_API_VERSION")
model = os.getenv("OPENAI_MODEL")
api_key = os.getenv("OPENAI_API_KEY")

url = f"{endpoint}openai/deployments/{model}/chat/completions?api-version={version}"

headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

formatting_prompt = get_formatting_prompt(markdown_table)
formatting_payload = {
    "messages": [
        {"role": "user", "content": formatting_prompt}
    ],
    "temperature": 0,
    "max_tokens": 50,
}

#saving as {PMC}\n---\n{prompt}\n---\n{response}
formatting_response_json = requests.post(url, headers=headers, json=formatting_payload)
time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
with open(f"responses/{time}_formatting.txt", "w", encoding="utf-8") as file:
    file.write(f"{PMC}\n---\n{formatting_prompt}\n---\n{formatting_response_json.json()['choices'][0]['message']['content']}")
print(formatting_response_json.status_code)

formatting_args = [brackets[1:-1] for brackets in formatting_response_json.json()['choices'][0]['message']['content'].split("\n")]

200


Alternatively, read in previous response

In [ ]:
response_file = "responses/2025-06-03_15-22-55_formatting.txt"
with open(f"{response_file}", "r", encoding="utf-8") as file:
    formatting_args = [brackets[1:-1] for brackets in file.read().split("\n---\n")[2].split("\n")]

In [ ]:
xml_content = html.unescape(html_content)
markdown_table = table_utils.single_html_table_to_markdown(xml_content)

print(markdown_table)
print("\n---\n")

if range(int(formatting_args[1])):
    markdown_table = table_utils.combine_markdown_group_cols(markdown_table, int(formatting_args[1]))
markdown_table = table_utils.remove_headers(markdown_table)
if formatting_args[0].lower() == "vertical":
    markdown_table = table_utils.combine_m_and_SD(markdown_table)
    markdown_table = table_utils.transpose_markdown_table(markdown_table)
markdown_table = table_utils.separate_m_and_SD(markdown_table)
for char in formatting_args[2].split("|"):
    if char == "" or char == " ":
        continue
    markdown_table = markdown_table.replace(f" {char.strip()}", "").replace(f"{char.strip()}", "")

print(markdown_table)

| Organ | Control group (g) | Treatment group (2000 mg/kg b.w) (g) |
| --- | --- | --- |
| Liver | 4.21 +- 0.25 | 4.85 +- 0.26 |
| Kidneys | 1.11 +- 0.04 | 1.29 +- 0.09 |
| Spleen | 0.61 +- 0.07 | 0.53 +- 0.04 |
| Heart | 0.48 +- 0.02 | 0.50 +- 0.05 |
| Lungs | 1.16 +- 0.08 | 1.11 +- 0.08 |
| Brain | 1.16 +- 0.04 | 1.20 +- 0.09 |
| Uterus | 2.34 +- 0.18 | 2.31 +- 0.28 |

---

| Organ | Kidneys.mean | Kidneys.SD | Spleen.mean | Spleen.SD | Heart.mean | Heart.SD | Lungs.mean | Lungs.SD | Brain.mean | Brain.SD |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Control group (g) | 1.11 | 0.04 | 0.61 | 0.07 | 0.48 | 0.02 | 1.16 | 0.08 | 1.16 | 0.04 |
| Treatment group (2000 mg/kg b.w) (g) | 1.29 | 0.09 | 0.53 | 0.04 | 0.50 | 0.05 | 1.11 | 0.08 | 1.20 | 0.09 |


c:\Users\jelli\Desktop\Animal Toxicity Trials\process_table\table_utils.py:410: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(html_content, "html.parser")
c:\Users\jelli\Desktop\Animal Toxicity Trials\process_table\table_utils.py:18: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser 

In [ ]:
df = table_utils.markdown_to_dataframe(markdown_table)
group_list = df.iloc[:,0].tolist()
with open(f"{PMC}.xml", "r", encoding="utf-8") as file:
    paper_xml = file.read()
soup = BeautifulSoup(paper_xml, "lxml-xml")
methods_passages = [p for p in soup.find_all("passage") if p.find("infon", {"key": "section_type"}) and 
        p.find("infon", {"key": "section_type"}).text.strip().upper() == "METHODS"]
methods_text_list = [p.find("text").get_text() for p in methods_passages if p.find("text")]
paper = "\n".join(methods_text_list)
with open(f"{PMC}_text.txt", "w", encoding="utf-8") as file:
    file.write(paper)
groups_prompt = get_groups_prompt(paper, group_list)
biomarkers_prompt = get_biomarker_prompt(paper, markdown_table)

API Call

In [ ]:
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
version = os.getenv("OPENAI_API_VERSION")
model = os.getenv("OPENAI_MODEL")
api_key = os.getenv("OPENAI_API_KEY")

url = f"{endpoint}openai/deployments/{model}/chat/completions?api-version={version}"
print(url)

headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

groups_payload = {
    "messages": [
        {"role": "user", "content": groups_prompt}
    ],
    "temperature": 0,
    "max_tokens": 1000,
}

biomarkers_payload = {
    "messages": [
        {"role": "user", "content": biomarkers_prompt}
    ],
    "temperature": 0,
    "max_tokens": 1000,
}

#saving as {PMC}\n---\n{prompt}\n---\n{response}
groups_response_json = requests.post(url, headers=headers, json=groups_payload)
biomarkers_response_json = requests.post(url, headers=headers, json=biomarkers_payload)
time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
with open(f"responses/{time}_group.txt", "w", encoding="utf-8") as file:
    file.write(f"{PMC}\n---\n{groups_prompt}\n---\n{groups_response_json.json()['choices'][0]['message']['content']}")
with open(f"responses/{time}_biomarkers.txt", "w", encoding="utf-8") as file:
    file.write(f"{PMC}\n---\n{biomarkers_prompt}\n---\n{biomarkers_response_json.json()['choices'][0]['message']['content']}")
print(groups_response_json.status_code)
print(biomarkers_response_json.status_code)

https://pharmacoinfo-openai-5.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-10-21
200
200


In [31]:
groups_response = groups_response_json.json()['choices'][0]['message']['content']
biomarkers_response = biomarkers_response_json.json()['choices'][0]['message']['content']
print(groups_response)
print("---")
print(biomarkers_response)


<Control(PBS)|mouse|BALB/c nude|BT-474|4|PBS|N/A|N/A|iv|N/A>  
<DM1(60 mug/kg)|mouse|BALB/c nude|BT-474|4|DM1|60|mug/kg|iv|three times every three days>  
<ApDC(3.85 mg/kg)|mouse|BALB/c nude|BT-474|4|ApDC|3.85|mg/kg|iv|three times every three days>  
---
<Hematology,10/muL.Platelet, counts.mean|3 days|Platelet counts|cardiovascular|10^3/μL|mean>  
<Hematology,10/muL.Platelet, counts.SD|3 days|Platelet counts|cardiovascular|10^3/μL|SD>  
<Hematology,10/muL.Red blood cells, M/muL.mean|3 days|Red blood cells|cardiovascular|10^6/μL|mean>  
<Hematology,10/muL.Red blood cells, M/muL.SD|3 days|Red blood cells|cardiovascular|10^6/μL|SD>  
<Hematology,10/muL.Hematocrits, %.mean|3 days|Hematocrits|cardiovascular|%|mean>  
<Hematology,10/muL.Hematocrits, %.SD|3 days|Hematocrits|cardiovascular|%|SD>  
<Hematology,10/muL.Neutrophil, absolute.mean|3 days|Neutrophil absolute|cardiovascular|10^3/μL|mean>  
<Hematology,10/muL.Neutrophil, absolute.SD|3 days|Neutrophil absolute|cardiovascular|10^3/μL|SD>

In [38]:
groups = [group.strip()[1:-1].split("|") for group in groups_response.split("\n")]
biomarkers = [marker.strip()[1:-1].split("|") for marker in biomarkers_response.split("\n")]
max_treatments = int((max(len(group) for group in groups) - 5)/5)
final_md = "| Animal Species | Model Type | Xenograft | Sample Size |" + "".join(f" Treatment {i+1} | Dose {i+1} | Dose Units {i+1} | Route {i+1} | Frequency {i+1} |" for i in range(max_treatments)) + " Terminal Time | Biomarker | System Affected | Numerical Value | Units | Summary Statistic |\n|" + " --- |" * (10+5*max_treatments) + "\n"
group_dict = {group: i for i, group in enumerate(df.iloc[:, 0])}
for group in groups:
    group_string = "| " + group[1] + " | " + group[2] + " | " + group[3] + " | " + group[4] + " | "
    group_string += "".join(group[i] + " | " for i in range(5, len(group)))
    group_string += "N/A | " * (max_treatments*5 - len(group) + 5)
    for biomarker in biomarkers:
        row = group_string
        row += biomarker[1] + " | "                                 #terminal time
        row += biomarker[2] + " | "                                 #biomarker (new_name)
        row += biomarker[3] + " | "                                 #system affected
        row += df.loc[group_dict[group[0]], biomarker[0]]  + " | "  #value
        row += biomarker[4] + " | "                                 #units
        row += biomarker[5] + " |\n"                                #summary statistic
        final_md += row
print(final_md)


| Animal Species | Model Type | Xenograft | Sample Size | Treatment 1 | Dose 1 | Dose Units 1 | Route 1 | Frequency 1 | Terminal Time | Biomarker | System Affected | Numerical Value | Units | Summary Statistic |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | Platelet counts | cardiovascular | 697.50 | 10^3/μL | mean |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | Platelet counts | cardiovascular | 64.16 | 10^3/μL | SD |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | Red blood cells | cardiovascular | 8.38 | 10^6/μL | mean |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | Red blood cells | cardiovascular | 0.27 | 10^6/μL | SD |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | Hematocrits | cardiovascular | 45.86 | % | mean |
| mouse | BALB/c nude | BT-474 | 4 |

| Animal Species | Model Type | Xenograft | Sample Size | Treatment 1 | Dose 1 | Dose Units 1 | Route 1 | Frequency 1 | Terminal Time | Biomarker | System Affected | Numerical Value | Units | Summary Statistic |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | Platelet counts | cardiovascular | 697.50 | 10^3/μL | mean |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | Platelet counts | cardiovascular | 64.16 | 10^3/μL | SD |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | Red blood cells | cardiovascular | 8.38 | 10^6/μL | mean |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | Red blood cells | cardiovascular | 0.27 | 10^6/μL | SD |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | Hematocrits | cardiovascular | 45.86 | % | mean |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | Hematocrits | cardiovascular | 2.37 | % | SD |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | Neutrophil absolute | cardiovascular | 0.54 | 10^3/μL | mean |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | Neutrophil absolute | cardiovascular | 0.09 | 10^3/μL | SD |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | Lymphocyte | cardiovascular | 2.22 | 10^3/μL | mean |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | Lymphocyte | cardiovascular | 0.50 | 10^3/μL | SD |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | ALT | digestive | 26.28 | U/L | mean |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | ALT | digestive | 6.33 | U/L | SD |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | AST | digestive | 89.46 | U/L | mean |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | AST | digestive | 17.86 | U/L | SD |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | ALP | digestive | 295.11 | U/L | mean |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | ALP | digestive | 99.65 | U/L | SD |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | GGT | digestive | 4.97 | U/L | mean |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | GGT | digestive | 1.00 | U/L | SD |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | Total bilirubin | digestive | 0.06 | U/L | mean |
| mouse | BALB/c nude | BT-474 | 4 | PBS | N/A | N/A | iv | N/A | 3 days | Total bilirubin | digestive | 0.03 | U/L | SD |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | Platelet counts | cardiovascular | 632.40 | 10^3/μL | mean |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | Platelet counts | cardiovascular | 101.64 | 10^3/μL | SD |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | Red blood cells | cardiovascular | 8.08 | 10^6/μL | mean |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | Red blood cells | cardiovascular | 0.35 | 10^6/μL | SD |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | Hematocrits | cardiovascular | 46.38 | % | mean |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | Hematocrits | cardiovascular | 1.41 | % | SD |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | Neutrophil absolute | cardiovascular | 0.37 | 10^3/μL | mean |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | Neutrophil absolute | cardiovascular | 0.15 | 10^3/μL | SD |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | Lymphocyte | cardiovascular | 1.78 | 10^3/μL | mean |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | Lymphocyte | cardiovascular | 0.89 | 10^3/μL | SD |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | ALT | digestive | 32.29 | U/L | mean |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | ALT | digestive | 14.33 | U/L | SD |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | AST | digestive | 89.20 | U/L | mean |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | AST | digestive | 11.66 | U/L | SD |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | ALP | digestive | 292.01 | U/L | mean |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | ALP | digestive | 84.44 | U/L | SD |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | GGT | digestive | 7.02 | U/L | mean |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | GGT | digestive | 2.11 | U/L | SD |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | Total bilirubin | digestive | 0.07 | U/L | mean |
| mouse | BALB/c nude | BT-474 | 4 | DM1 | 60 | mug/kg | iv | three times every three days | 3 days | Total bilirubin | digestive | 0.03 | U/L | SD |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | Platelet counts | cardiovascular | 425.33 | 10^3/μL | mean |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | Platelet counts | cardiovascular | 66.11 | 10^3/μL | SD |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | Red blood cells | cardiovascular | 7.69 | 10^6/μL | mean |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | Red blood cells | cardiovascular | 0.34 | 10^6/μL | SD |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | Hematocrits | cardiovascular | 43.17 | % | mean |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | Hematocrits | cardiovascular | 1.20 | % | SD |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | Neutrophil absolute | cardiovascular | 0.31 | 10^3/μL | mean |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | Neutrophil absolute | cardiovascular | 0.11 | 10^3/μL | SD |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | Lymphocyte | cardiovascular | 1.55 | 10^3/μL | mean |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | Lymphocyte | cardiovascular | 0.18 | 10^3/μL | SD |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | ALT | digestive | 42.93 | U/L | mean |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | ALT | digestive | 14.86 | U/L | SD |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | AST | digestive | 95.35 | U/L | mean |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | AST | digestive | 20.89 | U/L | SD |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | ALP | digestive | 211.76 | U/L | mean |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | ALP | digestive | 44.13 | U/L | SD |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | GGT | digestive | 6.28 | U/L | mean |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | GGT | digestive | 3.67 | U/L | SD |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | Total bilirubin | digestive | 0.07 | U/L | mean |
| mouse | BALB/c nude | BT-474 | 4 | ApDC | 3.85 | mg/kg | iv | three times every three days | 3 days | Total bilirubin | digestive | 0.02 | U/L | SD |



Isolating methods section and constructing prompts